In [1]:
import math
import os
import sys
sys.path.append("..")
import argparse
import string
import re

set global variables

In [2]:
epsilon1=['0.250000','0.350000','0.450000','0.550000','0.650000','0.750000']
epsilon2=['0.300000','0.400000','0.500000','0.600000','0.700000','0.800000']
mu=['2','5','10','15']

hypergraph='/home/hnu/Disk0/ParDon/hypergraph/vldb23NeighborCore/real/DBLP.hyp'
cluster_path='/home/hnu/Disk0/llw/my_cpp12_hash/output_lab1_csv/cluster_optimize_index_dblp'
output_file='dblp2.txt'

load hypergraph

In [3]:
def read_graph(hypergraph):
    n=0
    infile=open(hypergraph,'r')
    for line in infile:
        temp_string=line.strip('\n')
        if temp_string!='':
            n+=1
    infile.close()
    print(str(n))
    return n

load cluster result 1 (true labels)

In [4]:
def read_label(cluster1,n):
    c1=[]
    ed={}
    infile=open(cluster1,'r')
    for line in infile:
        temp_string=line.strip('\n')
        temp=re.split(',',temp_string)
        c={}
        for str_e in temp:
            if str_e!='':
                c[int(str_e)]='exist'
                ed[int(str_e)]='exist'
        c1.append(c)
    infile.close()
    print('true label num: '+str(len(c1)))
    c_temp={}
    for i in range(n):
        if i not in ed.keys():
            c_temp[i]='exist'
    c1.append(c_temp)
    return c1

load cluster result 2 (cluster labels)

In [5]:
def read_cluster(cluster2,n):
    c2=[]
    ed={}
    infile=open(cluster2,'r')
    for line in infile:
        temp_string=line.strip('\n')
        temp=re.split(',',temp_string)
        c=[]
        for str_e in temp:
            if str_e!='':
                c.append(int(str_e))
                ed[int(str_e)]='exist'
        c2.append(c)
    infile.close()
    print('cluster label num: '+str(len(c2)))
    c_temp=[]
    for i in range(n):
        if i not in ed.keys():
            c_temp.append(i)
    c2.append(c_temp)
    return c2

construct contingency table

In [6]:
def construct_table(c1,c2):
    contingency_table=[]
    sum_cluster_in_label=[]
    sum_label_in_cluster=[]
    for i in range(len(c1)):
        table=[]
        if i%1000==0 :
            print('i: '+str(i))
        for c in c2:
            sum=0
            for e in c:
                if e in c1[i].keys():
                    sum+=1
            table.append(sum)
        contingency_table.append(table)
    print('main table is constructed')

    for i in range(len(contingency_table)):
        sum=0
        for num in contingency_table[i]:
            sum+=num
        sum_label_in_cluster.append(sum)
    print('b list is constructed')

    for i in range(len(contingency_table[0])):
        sum=0
        for j in range(len(contingency_table)):
            sum+=contingency_table[j][i]
        sum_cluster_in_label.append(sum)
    print('a list is constructed')
    return contingency_table,sum_cluster_in_label,sum_label_in_cluster

compute ARI

In [7]:
def compute_ari(contingency_table,sum_label_in_cluster,sum_cluster_in_label,n,output_file,mu,epsilon):
    sum_c_nij_2=0
    for i in range(len(contingency_table)):
        for j in range(len(contingency_table[i])):
            sum_c_nij_2+=math.comb(contingency_table[i][j],2)

    sum_c_ai_2=0
    for i in range(len(sum_label_in_cluster)):
        sum_c_ai_2+=math.comb(sum_label_in_cluster[i],2)

    sum_c_bj_2=0
    for i in range(len(sum_cluster_in_label)):
        sum_c_ai_2+=math.comb(sum_cluster_in_label[i],2)

    c_n_2=math.comb(n,2)

    d1=sum_c_nij_2-((sum_c_ai_2*sum_c_bj_2)/c_n_2)
    d2=(sum_c_ai_2+sum_c_bj_2)/2-((sum_c_ai_2*sum_c_bj_2)/c_n_2)

    ari=d1/d2
    print('ARI: '+str(ari))

    ofile=open(output_file,'a')
    ofile.write('epsilon:'+epsilon+' mu:'+mu+' ARI: '+str(ari))
    ofile.write('\n')
    ofile.close()

In [8]:
def fast_ari(c1,c2,n,output_file,mu,epsilon):
    sum_cluster_in_label=[]
    for i in range(len(c2)):
        sum_cluster_in_label.append(0)

    sum_c_nij_2=0
    sum_c_ai_2=0
    for i in range(len(c1)):
        if i%1000==0 :
            print('i: '+str(i))
        sum_label_in_cluster=0
        for j in range(len(c2)):
            sum=0
            for e in c2[j]:
                if e in c1[i].keys():
                    sum+=1
            sum_c_nij_2+=math.comb(sum,2)
            sum_label_in_cluster+=sum
            sum_cluster_in_label[j]+=sum
        sum_c_ai_2+=math.comb(sum_label_in_cluster,2)

    print('main table is constructed')

    sum_c_bj_2=0
    for i in range(len(sum_cluster_in_label)):
        sum_c_ai_2+=math.comb(sum_cluster_in_label[i],2)

    c_n_2=math.comb(n,2)

    d1=sum_c_nij_2-((sum_c_ai_2*sum_c_bj_2)/c_n_2)
    d2=(sum_c_ai_2+sum_c_bj_2)/2-((sum_c_ai_2*sum_c_bj_2)/c_n_2)

    ari=d1/d2
    print('ARI: '+str(ari))
    ofile=open(output_file,'a')
    ofile.write('epsilon:'+epsilon+' mu:'+mu+' ARI: '+str(ari))
    ofile.write('\n')
    ofile.close()

In [9]:
n=read_graph(hypergraph)
for i in range(len(epsilon1)):
    for j in range(len(mu)):
        cluster1=cluster_path+'_'+mu[j]+'_'+epsilon1[i]+'.csv'
        cluster2=cluster_path+'_'+mu[j]+'_'+epsilon2[i]+'.csv'
        c1=read_label(cluster1,n)
        c2=read_cluster(cluster2,n)
        # contingency_table,sum_cluster_in_label,sum_label_in_cluster=construct_table(c1,c2)
        # compute_ari(contingency_table,sum_label_in_cluster,sum_cluster_in_label,n,output_file,mu[j],epsilon1[i])
        fast_ari(c1,c2,n,output_file,mu[j],epsilon1[i])

2170260
true label num: 11832
cluster label num: 16786
i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
i: 9000
i: 10000
i: 11000
main table is constructed
b list is constructed
a list is constructed
i: 0
i: 1000
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hnu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_370027/3047246374.py", line 10, in <module>
    fast_ari(c1,c2,n,output_file,mu[j],epsilon1[i])
  File "/tmp/ipykernel_370027/2229351682.py", line -1, in fast_ari
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hnu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hnu/anaconda3/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hnu/anaconda3/lib/python3.11/site-packages/